<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GPT4_AI_FLIGHT_PLAN_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython import get_ipython
from IPython.display import display

In [2]:
!nvidia-smi

Thu Jan  9 02:10:40 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   55C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
# Install required libraries
!pip install --upgrade datasets accelerate evaluate --quiet
!pip install trl --quiet
!pip install openai --quiet
!pip install requests --quiet # install requests
!pip install python-dotenv --quiet # install python-dotenv

# Install colab-env
!pip install colab-env -q

!pip install faiss-gpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 27.5 MB/s eta 0:00:00


In [11]:
import os
from dotenv import load_dotenv
import random
import faiss
import numpy as np
import openai # explicitly import openai
from openai import OpenAI


# Load the .env file
load_dotenv()

# Set your OpenAI API key
# get the api key from the environment.
openai_api_key = os.environ.get("OPENAI_API_KEY")
if openai_api_key is None:
     raise ValueError("OPENAI_API_KEY environment variable not set.")

# Initialize the OpenAI client
client = OpenAI(api_key=openai_api_key)


# Model ID for GPT-4
model_id = "gpt-4"

# Initialize Faiss index
embedding_size = 1536  # Adjust based on the model's output
index = faiss.IndexFlatL2(embedding_size)

# Cache for storing generated flight plans and their embeddings
flight_plan_cache = {}


def get_embedding(text):
        """Gets the embedding of the text using the new OpenAI API."""
        # Replace openai.Embedding.create with openai.client.embeddings.create
        # Use the appropriate embedding model (e.g., text-embedding-3-small)
        response = client.embeddings.create(input=text, model="text-embedding-3-small") # use the client object
        embedding = response.data[0].embedding
        return np.array(embedding, dtype=np.float32)


def generate_response(prompt, max_new_tokens=1024):
    """Generates a response using GPT-4, possibly using cached results."""

    # Calculate embedding for the prompt
    embedding = get_embedding(prompt)

    # Search for similar flight plans in the index
    D, I = index.search(
        embedding.reshape(1, -1), k=1
    )  # Search for nearest neighbor

    # Set a similarity threshold (adjust as needed)
    similarity_threshold = 0.2
    if D[0][0] < similarity_threshold:
        print("Using similar flight plan from memory...")
        similar_prompt = list(flight_plan_cache.keys())[I[0][0]]
        return flight_plan_cache[similar_prompt]

    print("Generating new flight plan...")

    messages = [{"role": "user", "content": prompt}]

    # Generate output using the OpenAI API
    try:
        response = client.chat.completions.create(  # Correct API call
            model=model_id,
            messages=messages,
            max_tokens=max_new_tokens,
            temperature=0.7,  # Adjust temperature as needed
        )

    except Exception as e:
        return f"An error occurred: {e}"

    # Extract the text of the response
    if response is None or not response.choices:
        response_text = "The model could not generate a response for your prompt."
    else:
        response_text = response.choices[0].message.content  # Access content

    # Store in the cache and Faiss index
    flight_plan_cache[prompt] = response_text
    index.add(embedding.reshape(1, -1))
    return response_text


def get_simulated_weather(location):
    """Simulates weather data for a given location."""
    # Define some possible weather conditions and values
    weather_conditions = ["Clear", "Cloudy", "Rainy", "Snowy", "Foggy"]
    temperature_ranges = {
        "New York": (-5, 30),  # Example range in Celsius
        "New York City": (-5, 30),
        "new york city": (-5, 30),
        "Shanghai": (0, 35),  # Example range in Celsius
        "shanghai": (0, 35),
    }
    wind_speed_range = (0, 20)  # Example range in m/s
    humidity_range = (30, 100)  # Example range in %

    # Choose random values based on the location
    condition = random.choice(weather_conditions)
    temperature_min, temperature_max = temperature_ranges.get(location, (0, 25))
    temperature = round(random.uniform(temperature_min, temperature_max), 1)
    feels_like = round(temperature + random.uniform(-2, 2), 1)
    wind_speed = round(random.uniform(*wind_speed_range), 1)
    humidity = random.randint(*humidity_range)

    # Format the weather information
    weather_info = (
        f"Simulated weather in {location}:\n"
        f"Temperature: {temperature}°C (feels like {feels_like}°C)\n"
        f"Conditions: {condition}\n"
        f"Humidity: {humidity}%\n"
        f"Wind speed: {wind_speed} m/s"
    )

    return weather_info


def create_flight_plan(
    origin,
    destination,
    aircraft_type,
    flight_date,
    preferred_route=None,
    cruising_altitude=None,
    departure_time=None,
):
    """Creates a flight plan using GPT-4."""

    prompt = f"""
    ## Flight Plan Request:

    **Origin:** {origin}
    **Destination:** {destination}
    **Aircraft Type:** {aircraft_type}
    **Flight Date:** {flight_date}
    """

    if preferred_route:
        prompt += f"**Preferred Route:** {preferred_route}\n"
    if cruising_altitude:
        prompt += f"**Cruising Altitude:** {cruising_altitude}\n"
    if departure_time:
        prompt += f"**Departure Time:** {departure_time}\n"

    prompt += """

    **Please generate a detailed flight plan including:**

    * **Route:** with waypoints (if applicable)
    * **Estimated Flight Duration:**
    * **Fuel Requirements:** (estimated)
    * **Cruising Altitude:** (if not specified)
    * **Departure Time:** (if not specified)
    * **Estimated Arrival Time:**
    * **Alternate Airports:** (if applicable)
    * **Fuel Consumption:** (estimated) in kg/hour

    **Format the flight plan clearly for readability.**
    """

    # Generate the flight plan (using the cache-aware generate_response)
    flight_plan = generate_response(prompt)
    return flight_plan


# --- Agent framework ---


class FlightPlanAgent:
    def __init__(self):
        self.memory = {"previous_flight_plans": {}}

    def extract_flight_info(self, user_input):
        """
        Extracts origin and destination from user input, handling multi-word locations.
        """
        origin = None
        destination = None
        words = user_input.lower().split()

        if "from" in words and "to" in words:
            from_index = words.index("from")
            to_index = words.index("to")

            # Extract the origin (multi-word)
            origin_words = []
            for i in range(from_index + 1, to_index):
                origin_words.append(words[i])
            origin = " ".join(origin_words)

            # Extract the destination (multi-word)
            destination_words = words[to_index + 1:]
            destination = " ".join(destination_words)

        # Force capitalization to match dictionary keys.
        if origin is not None:
            origin = " ".join(word.capitalize() for word in origin.split())
            origin = origin.strip()  # Remove spaces.
        if destination is not None:
            destination = " ".join(word.capitalize() for word in destination.split())
            destination = destination.strip()  # Remove spaces.

        return origin, destination

    def get_weather(self, location):
        """
        This now simulates weather information.
        """
        return get_simulated_weather(location)

    def interact(self, user_input):
        origin, destination = self.extract_flight_info(user_input)

        if origin and destination:
            # Check for existing flight plan in memory
            key = f"{origin}-{destination}"
            if key in self.memory["previous_flight_plans"]:
                previous_plan = self.memory["previous_flight_plans"][key]
                return (
                    f"I found a previous flight plan from {origin} to {destination}: \n\n"
                    f"{previous_plan}\n\n"
                    "Would you like to create a new one?"
                )

            # Get weather information (using the weather API)
            origin_weather = self.get_weather(origin)
            destination_weather = self.get_weather(destination)

            # Generate a new flight plan
            flight_plan = create_flight_plan(
                origin,
                destination,
                "Boeing 777",
                "2024-01-15",  # You can add more parameters
            )
            self.memory["previous_flight_plans"][key] = flight_plan
            return (
                f"Here's a flight plan from {origin} to {destination}: \n\n"
                f"{flight_plan}\n\n"
                f"Weather in {origin}: {origin_weather}\n"
                f"Weather in {destination}: {destination_weather}"
            )
        else:
            return "I can create flight plans. Please specify origin and destination (e.g., 'flight plan from New York to Shanghai')"


# Create the agent
agent = FlightPlanAgent()

# Interaction loop
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    response = agent.interact(user_input)
    print(f"Agent: {response}")

You: flight path from new york city to shanghai
Generating new flight plan...
Agent: Here's a flight plan from New York City to Shanghai: 

## Briefing for Flight Plan: 

**Origin:** John F. Kennedy International Airport (JFK), New York City, USA
**Destination:** Shanghai Pudong International Airport (PVG), Shanghai, China

**Aircraft:** Boeing 777-200LR

---
### **Route:**
- Departure from JFK Airport
- Take off from Runway 31L, heading northeast
- Enter J70 Airway at waypoint PUT (Putnam)
- Continue onto J547 Airway at waypoint YOW (Ottawa)
- Enter NAT (North Atlantic Tracks) at waypoint 5950N
- Cross into European airspace at waypoint DUB (Dublin), enter UL975 Airway
- Continue on L980 Airway at waypoint BELOX
- Enter A87 Airway at waypoint GIGOR
- Continue on B210 Airway at waypoint AKADA
- Enter the Chinese airspace at waypoint MORIT on A575 Airway
- Arrival at PVG Airport, landing on Runway 34R

### **Estimated Flight Duration:**
Approximately 15 hours, 30 minutes

### **Fuel Req